# Roteamento para empresa *Shop+*

### Solução de roteamento para entrega de mercadorias para empresa *Shop+*

Disciplina: *Algoritmos em Grafos (GCC218)*

Docente: Mayron César de Oliveira Moreira

Semestre: 2018/2

Alunos: 
* Gabrielle Almeida Cuba
* Guilherme Barbosa Ochikubo
* Guilherme Henrique de Melo
* João Paulo Pena
* Leonardo Henrique de Braz
* Ruan Marcos Basílio

## Definições
### Introdução e Problema

A empresa * Shop+ * é uma grande varejista nacional e possui um problema ainda não satisfatoriamente resolvido. O custo de transporte de suas mercadorias constitui o gargalo da empresa, tanto para as vendas em loja física quanto para *e-commerce*. 
    
Após um meticuloso estudo realizado pelo departamento financeiro da *Shop+*, foi constatado que a contratação de uma empresa de consultoria seria necessária para que a *Shop+* economizasse na entrega de seus produtos e, assim, pudesse investir o valor poupado na modernização do próprio negócio. A empresa necessita, em poucas palavras, de um sistema que diariamente lhe dê percursos otimizados para o atendimento da demanda de seus clientes.


### Especificações
* Inicialmente, para testar a eficácia deste sistema, a diretoria da Shop+ escolheu uma grande cidade na qual a empresa possui sede, composta por 5 centros de distribuição;
* Segundo o gerente da Shop+, a realização das entregas de mercadorias nesta cidade é feita por uma frota heterogênea, composta de carros, vans (próprias e terceirizadas), mini-vans e motocicletas;
* Por questões legais, os motoristas têm jornadas diárias de 7 horas de trabalho;
* A capacidade de cada tipo de veículo é limitada pelo volume em m3 de pacotes que este é capaz de transportar; 
* O valor monetário em mercadorias que cada veículo leva em sua viagem deve ser superiormente limitado.

### Considerações do problema

Para solução, foram tomadas algumas decisões:

* Como a empresa define os centros de distribuições, o emprego de demanda é definido e distribuido de forma balanceada entre os centros de distribuições. Ou seja, cada centro de distribuição possui entregas balanceadas pela quantidade de clientes que deverão ser atendidos e também pelo volume de encomendas a serem entregues pelos centros
* O roteamento dos veículos para transporte deve ser dado com ênfase em redução de custos, alocados entre os centros de distribuição. Cada veículo poderá apenas ser utilizado em uma rota, sem rodízio de entregas. A somatória de gastos em rotas e o valor de alocação dos carros definem o preço a ser gasto nas entregas;

## Solução do Problema

Sejam os centros de distribuições definidos no arquivo abaixo:


In [ ]:
caminho_arquivo = input("Digite o caminho para o arquivo de dados: ")

# Modelagem do grafo

   Inicialmente, foi criada uma classe Cliente para a modelagem do grafo, onde todo cliente é um vértice, uma vez que tal vértíce tenha o atributo centro = True, significa que este é um centro de distribuiçao.

In [ ]:
from math import ceil
from math import hypot

class Cliente(object):
    def __init__(self, v, valor, qtdPacotes, coordX, coordY):
        self.volume = v
        self.__valor = valor
        self.pacotes = qtdPacotes
        self.__coordX = coordX
        self.__coordY = coordY
        self.centro = False
        self.soma_distancias = 0

    def getValor(self):
        return self.__valor

    def getX(self):
        return self.__coordX  

    def getY(self):
        return self.__coordY 

    def __repr__(self):
        if self.centro:
            return "({0}, {1}, {2}, {3})".format(ceil(self.__coordX), ceil(self.__coordY), ceil(self.volume), self.pacotes)
        else:    
            return "({0}, {1})".format(ceil(self.__coordX), ceil(self.__coordY))


# Definição de distância inicial

Após a modelagem dos vértices, é calculada a distância entre pontos de todos os vértices não centro(Clientes) aos vértices centro,cada cliente é associado ao centro de distribuição mais próximo e é criada uma aresta de ligação entre eles, onde o peso da aresta representa a distância entre os pontos.

In [ ]:
import networkx as nx
import math 
import random
import matplotlib.pyplot as a
import matplotlib as mpl
import copy
from Grafo import MontaCaminhos

from modelo.clientes import Cliente
from modelo.veiculo import Veiculo

def intersecao(centro, cliente, centro1, cliente1):
    det = (cliente1.getX() - centro1.getX()) * (cliente.getY() - centro.getY())  -  (cliente1.getY() - centro1.getY()) * (cliente.getX() - centro.getX())
    return det != 0

def existe_intersecao(centro, cliente, centro1, cliente1):
    return ((centro.getY() >= centro1.getY()) == (cliente.getY() <= cliente1.getY())) and ((centro.getY() <= centro1.getY()) == (cliente.getY() >= cliente1.getY()))


def distancia(c1, c2):
    return math.sqrt(
        ((c2.getX() - c1.getX()) ** 2) +
        ((c2.getY() - c1.getY()) ** 2))
        
def ler_cliente(x, y, volume, valor, pacotes,flag):
    return Cliente(volume, valor, pacotes, x, y)

def ler_veiculo(V, P, Nv, vf, vd, tc, td, ph, pkm, pf):
    return Veiculo(V, P, Nv, vf, vd, tc, td, ph, pkm, pf) 

def imprime_grafo(regioes, nome_arquivo, limite):
    a.clf()
    centros = list(regioes.keys())

    completo = nx.union(regioes[centros[0]], regioes[centros[1]])
    completo = nx.union(completo, regioes[centros[2]])
    completo = nx.union(completo, regioes[centros[3]])
    completo = nx.union(completo, regioes[centros[4]])

    mapeamento_x = []
    mapeamento_y = []

    for cliente in completo.nodes():
        if cliente not in centros:
            mapeamento_x.append(cliente.getX())
            mapeamento_y.append(cliente.getY())

    mpl.rc("axes", edgecolor="blue")
    a.plot(mapeamento_x, mapeamento_y, "ro")

    mapeamento_x = []
    mapeamento_y = []

    for centro in centros:
        mapeamento_x.append(centro.getX())
        mapeamento_y.append(centro.getY())

    a.plot(mapeamento_x, mapeamento_y, "bo")

    mapeamento_x = []
    mapeamento_y = []

    for centro in centros:
        for u, v in regioes[centro].edges():
            a.plot([u.getX(), v.getX()], [u.getY(), v.getY()], "--k")
    a.axis([0, limite, 0, limite])

    a.savefig(nome_arquivo)

nome_arquivo = input("Informe o arquivo de entrada: ")

arquivo = open("docs/{0}".format(nome_arquivo), "r")

qtd_casas = int(arquivo.readline())
qtd_centros = int(arquivo.readline())
qtd_veiculos = int(arquivo.readline())
qtd_horas = int(arquivo.readline())

# para cada centro de distribuição, existe um grafo
regioes = {}

clientes = []
veiculos = []

for i in range(qtd_centros):
    leitura = arquivo.readline().split()
    cliente = ler_cliente(float(leitura[0]), float(leitura[1]), float(leitura[2]), float(leitura[3]), float(leitura[4]), True)
    cliente.centro = True
    cliente.maior_distancia = 0 - float("inf")

    regioes[cliente] = nx.Graph()
    regioes[cliente].add_node(cliente, pos = (cliente.getX(), cliente.getY()))

for i in range(qtd_casas - qtd_centros):
    leitura = arquivo.readline().split()
    clientes.append(ler_cliente(float(leitura[0]), float(leitura[1]), float(leitura[2]), float(leitura[3]), float(leitura[4]), False))

for i in range(qtd_veiculos):
    leitura = arquivo.readline().split()    
    veiculos.append(ler_veiculo(float(leitura[0]), float(leitura[1]), float(leitura[2]), float(leitura[3]),
    float(leitura[4]), float(leitura[5]), float(leitura[6]), float(leitura[7]), float(leitura[8]), float(leitura[9])))

volume_total = 0
distancia_total = 0

# definição a partir de proximidade
for cliente in clientes: 
    menor_centro = list(regioes.keys())[0]
    menor_distancia = distancia(cliente, menor_centro)
    
    for centro in regioes.keys():
        if (distancia(cliente, centro) < distancia(cliente, menor_centro)):
            menor_centro = centro
            menor_distancia = distancia(cliente, centro)

    regioes[menor_centro].add_node(cliente)
    regioes[menor_centro].add_edge(cliente, menor_centro, distancia=menor_distancia)
    menor_centro.volume += cliente.volume
    menor_centro.pacotes += cliente.pacotes
    
    menor_centro.soma_distancias += distancia(cliente, menor_centro)

    # encontra a maior distancia de cliente e centro
    if menor_centro.maior_distancia < menor_distancia:
        menor_centro.maior_distancia = menor_distancia

    volume_total += cliente.volume

imprime_grafo(regioes, "exibicao/{0}sem-melhoria.png".format(nome_arquivo), qtd_casas)